In [5]:
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from datetime import timedelta, datetime

In [6]:
def scroll_to_bottom(driver, pause_time=2):
    """Desplazarse poco a poco hasta el final de la página."""
    last_height = driver.execute_script("return document.body.scrollHeight*0.9")
    
    while True:
        # Desplazarse hacia abajo
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*0.9);")
        
        # Esperar a que se cargue el contenido
        time.sleep(pause_time)
        
        # Calcular la nueva altura de la página y comparar con la última altura
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [45]:
urls = ['https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/',]
        #'https://www.oddsportal.com/basketball/usa/nba-2022-2023/results/',
        #'https://www.oddsportal.com/basketball/usa/nba-2021-2022/results/',
        #'https://www.oddsportal.com/basketball/usa/nba-2020-2021/results/',
        #'https://www.oddsportal.com/basketball/usa/nba-2019-2020/results/']
pages = [28,]
       #  28,
        # 28,
         #28,
         #28]
Nombres_ligas = ['NBA']
                # 'NBA'
                 #'NBA'
                 #'NBA'
                 #'NBA']

In [46]:
partidos = []
home_team = []
away_team = []
fechas = []
i = 0

for url in urls:
    temp_urls = [url+f'/#/page/{j}/' for j in range(1,pages[i]+1)]

    options = Options()
    #options.add_experimental_option("detach", True)
    options.add_argument("--disable-search-engine-choice-screen")

    j = 0
    while j <= pages[i]-1:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
        driver.get(temp_urls[j])

        driver.maximize_window()
        time.sleep(3)
        scroll_to_bottom(driver, pause_time=3)
        time.sleep(3)
        html = driver.page_source
        
        soup = BeautifulSoup(html, 'html.parser')
        
        divs = soup.find_all('div', class_='eventRow flex w-full flex-col text-xs')

        for div in divs:
            
            fecha_div = div.find('div', class_='text-black-main font-main w-full truncate text-xs font-normal leading-5')
            
            if fecha_div is not None:
                fecha_div = fecha_div.text
                fecha = fecha_div.split('-')[0].rstrip()
                if 'Today' in fecha:
                    fecha = datetime.now().strftime("%d %b %Y")
                elif 'Yesterday' in fecha:
                    fecha = (datetime.now() - timedelta(days=1)).strftime("%d %b %Y")
            divs2 = div.find_all('a', class_='next-m:flex next-m:!mt-0 ml-2 mt-2 min-h-[32px] w-full hover:cursor-pointer')

            for div2 in divs2:
                
                partidos.append('https://www.oddsportal.com' + div2['href'])
                equipos = div2.find_all('p', class_='participant-name truncate')
                home_team.append(equipos[0].text)
                away_team.append(equipos[1].text)
                fechas.append(fecha)
        j += 1        
        driver.quit()    

temp_data_df = pd.DataFrame()       
temp_data_df['URL'] = partidos
temp_data_df['H_TEAM_NICKNAME'] = home_team
temp_data_df['A_TEAM_NICKNAME'] = away_team
temp_data_df['GAME_DATE'] = fechas

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=134.0.6998.89)
Stacktrace:
	GetHandleVerifier [0x010BC7F3+24435]
	(No symbol) [0x01042074]
	(No symbol) [0x00F106E3]
	(No symbol) [0x00EFFEC0]
	(No symbol) [0x00F1DFDF]
	(No symbol) [0x00F846AF]
	(No symbol) [0x00F9EB19]
	(No symbol) [0x00F7D5B6]
	(No symbol) [0x00F4C54F]
	(No symbol) [0x00F4D894]
	GetHandleVerifier [0x013C70A3+3213347]
	GetHandleVerifier [0x013DB0C9+3295305]
	GetHandleVerifier [0x013D558C+3271948]
	GetHandleVerifier [0x01157360+658144]
	(No symbol) [0x0104B27D]
	(No symbol) [0x01048208]
	(No symbol) [0x010483A9]
	(No symbol) [0x0103AAC0]
	BaseThreadInitThunk [0x7602FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x76EC82AE+286]
	RtlGetAppContainerNamedObjectPath [0x76EC827E+238]


In [14]:
dataset_ods.duplicated().sum()

0

In [ ]:
dataset_ods = pd.DataFrame()
dataset_urls = temp_data_df.copy()

keys = ['URL', 'H_TEAM_NICKNAME', 'A_TEAM_NICKNAME', 'id', 'GAME_DATE', 'Average_1','Average_X','Average_2','Highest_1','Highest_X','Highest_2', 'Average_H', 'Average_A', 'Highest_H', 'Highest_A', 'Average_1X','Average_12','Average_X2','Highest_1X','Highest_12','Highest_X2']
extensiones = ['#1X2;3', '#home-away;1', '#double;3']
columnas = [['Average_1','Average_X','Average_2','Highest_1','Highest_X','Highest_2'],['Average_H', 'Average_A', 'Highest_H', 'Highest_A'],
        ['Average_1X','Average_12','Average_X2','Highest_1X','Highest_12','Highest_X2']]
dataset_urls['id'] = dataset_urls['GAME_DATE'].astype(str) + '_' + dataset_urls['H_TEAM_NICKNAME'].astype(str) + '_' + dataset_urls['A_TEAM_NICKNAME'].astype(str)

options = Options()
#options.add_experimental_option("detach", True)
options.add_argument("--disable-search-engine-choice-screen")
options.add_argument("--headless")

for index, row in dataset_urls.iterrows():

    if len(dataset_ods) == 0 or row['id'] not in dataset_ods['id'].values:

        print(index)
        diccionario_estadisticas = {key: None for key in keys}
        diccionario_estadisticas['URL'] = row['URL']
        diccionario_estadisticas['H_TEAM_NICKNAME'] = row['H_TEAM_NICKNAME']
        diccionario_estadisticas['A_TEAM_NICKNAME'] = row['A_TEAM_NICKNAME']
        diccionario_estadisticas['id'] = row['id']
        diccionario_estadisticas['GAME_DATE'] = row['GAME_DATE']
        i = -1
        for extension in extensiones:
            
            try:
                i += 1
                url = row['URL'] + extension

                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)

                driver.get(url)
                driver.maximize_window()
                time.sleep(3)
                last_height = driver.execute_script("return document.body.scrollHeight")
                while True:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(3)  # Espera un poco para que se cargue el contenido dinámico
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height
                    
                html = driver.page_source

                soup = BeautifulSoup(html, 'html.parser')

                if i == 0 or i == 1 or i == 2:
                    divs = soup.find_all('div', class_='border-black-borders flex min-w-[60px] flex-col items-center justify-center gap-1 border-l')
                    temp = []
                    for div in divs:
                        
                        if len(str(div)) == 240 or len(str(div)) == 241:
                            if div.find('p', class_='height-content').text[-1] != "%":
                                temp.append(div)
                    for j in range(len(temp)):
                        diccionario_estadisticas[columnas[i][j]] = temp[j].find('p', class_='height-content').text
            except Exception as e:
                # Si ocurre un error, se captura la excepción y se imprime la variable
                print(f"Ocurrió un error: {e} en el partido {index}")

    data_df = pd.DataFrame([diccionario_estadisticas])
    # Aqui debo incluir el formateo para que todo funcione correctamente
    dataset_ods = pd.concat([dataset_ods, data_df], ignore_index=True)
    #dataset_ods.to_csv('Datos raw/Datos_cuotas.csv',index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [51]:
dataset_ods

,liga,url,Home_team,Away_team,id,fecha,1_Average,X_Average,2_Average,1_Highest,...,H_Average,A_Average,H_Highest,A_Highest,1X_Average,12_Average,X2_Average,1X_Highest,12_Highest,X2_Highest
0,NBA,https://www.oddsportal.com/basketball/usa/nba-...,Boston Celtics,Dallas Mavericks,18 Jun 2024_NBA_Boston Celtics_Dallas Mavericks,18 Jun 2024,1.47,15.00,2.83,1.50,...,1.36,3.29,1.39,3.31,1.37,1.03,2.60,1.37,1.03,2.60
1,NBA,https://www.oddsportal.com/basketball/usa/nba-...,Dallas Mavericks,Boston Celtics,15 Jun 2024_NBA_Dallas Mavericks_Boston Celtics,15 Jun 2024,1.83,15.00,2.10,1.83,...,1.99,1.85,2.00,1.89,1.69,1.04,1.87,1.69,1.04,1.87
2,NBA,https://www.oddsportal.com/basketball/usa/nba-...,Dallas Mavericks,Boston Celtics,13 Jun 2024_NBA_Dallas Mavericks_Boston Celtics,13 Jun 2024,1.54,15.50,2.55,1.57,...,1.69,2.21,1.72,2.25,1.46,1.03,2.40,1.46,1.03,2.40
3,NBA,https://www.oddsportal.com/basketball/usa/nba-...,Boston Celtics,Dallas Mavericks,10 Jun 2024_NBA_Boston Celtics_Dallas Mavericks,10 Jun 2024,1.47,15.50,2.83,1.50,...,None,None,None,None,1.37,1.03,2.60,1.37,1.03,2.60
4,NBA,https://www.oddsportal.com/basketball/usa/nba-...,Boston Celtics,Dallas Mavericks,07 Jun 2024_NBA_Boston Celtics_Dallas Mavericks,07 Jun 2024,1.49,15.50,2.73,1.50,...,1.41,2.99,1.43,3.05,1.40,1.03,2.60,1.40,1.03,2.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4350,NBA,https://www.oddsportal.com/basketball/usa/nba-...,Toronto Raptors,Utah Jazz,03 Oct 2022_NBA_Toronto Raptors_Utah Jazz,03 Oct 2022,1.59,16.00,2.49,1.59,...,1.41,2.89,1.43,3.00,None,None,None,None,None,None
4351,NBA,https://www.oddsportal.com/basketball/usa/nba-...,Boston Celtics,Charlotte Hornets,02 Oct 2022_NBA_Boston Celtics_Charlotte Hornets,02 Oct 2022,1.55,16.00,2.59,1.55,...,1.40,2.93,1.45,3.10,None,None,None,None,None,None
4352,NBA,https://www.oddsportal.com/basketball/usa/nba-...,Washington Wizards,Golden State Warriors,02 Oct 2022_NBA_Washington Wizards_Golden Stat...,02 Oct 2022,2.18,16.00,1.74,2.18,...,2.42,1.58,2.55,1.65,2.40,14.50,1.66,2.40,14.50,1.66
4353,NBA,https://www.oddsportal.com/basketball/usa/nba-...,Milwaukee Bucks,Memphis Grizzlies,02 Oct 2022_NBA_Milwaukee Bucks_Memphis Grizzlies,02 Oct 2022,1.59,16.00,2.46,1.59,...,1.48,2.64,1.53,2.90,None,None,None,None,None,None


In [ ]:
dataset_ods.to_csv('Datos_cuotas.csv',index=False)

In [12]:
import pandas as pd
pd.set_option('display.float_format', '{:.0f}'.format)
EQUIPOS_ODDS = {
    "Oklahoma City Thunder": 1610612760,
    "Boston Celtics": 1610612738,
    "Cleveland Cavaliers": 1610612739,
    "Los Angeles Lakers": 1610612747,
    "Denver Nuggets": 1610612743,
    "Golden State Warriors": 1610612744,
    "New York Knicks": 1610612752,
    "Milwaukee Bucks": 1610612749,
    "Memphis Grizzlies": 1610612763,
    "Minnesota Timberwolves": 1610612750,
    "Los Angeles Clippers": 1610612746,
    "Indiana Pacers": 1610612754,
    "Houston Rockets": 1610612745,
    "Detroit Pistons": 1610612765,
    "Sacramento Kings": 1610612758,
    "Chicago Bulls": 1610612741,
    "Phoenix Suns": 1610612756,
    "Orlando Magic": 1610612753,
    "Atlanta Hawks": 1610612737,
    "Miami Heat": 1610612748,
    "Toronto Raptors": 1610612761,
    "Philadelphia 76ers": 1610612755,
    "Portland Trail Blazers": 1610612757,
    "San Antonio Spurs": 1610612759,
    "Brooklyn Nets": 1610612751,
    "Dallas Mavericks": 1610612742,
    "Charlotte Hornets": 1610612766,
    "Washington Wizards": 1610612764,
    "New Orleans Pelicans": 1610612740,
    "Utah Jazz": 1610612762,
}

# Aplicar el diccionario de mapeo a las columnas Home_team y Away_team
dataset_ods['H_TEAM_ID'] = dataset_ods['H_TEAM_NICKNAME'].map(EQUIPOS_ODDS).replace({float('nan'): None})
dataset_ods['A_TEAM_ID'] = dataset_ods['A_TEAM_NICKNAME'].map(EQUIPOS_ODDS).replace({float('nan'): None})
dataset_ods['GAME_DATE'] = pd.to_datetime(dataset_ods['GAME_DATE'])

# Mostrar el DataFrame resultante
type(dataset_ods.loc[0]['GAME_DATE'])

pandas._libs.tslibs.timestamps.Timestamp

In [39]:
import sqlite3
conexion = sqlite3.connect("NBA_DATA.db")

# Crear una lista para almacenar los GAME_IDs
game_ids = []

# Realizar una consulta SQL para cada fila en dataset_ods
for index, row in dataset_ods.iterrows():
    if row['H_TEAM_ID'] is None or row['A_TEAM_ID'] is None:
        game_ids.append(None)
        continue
    home_team_id = row['H_TEAM_ID']
    away_team_id = row['A_TEAM_ID']
    game_date = row['GAME_DATE']
    start_date = (game_date - pd.Timedelta(days=1)).strftime('%Y-%m-%d %H:%M:%S')
    game_date = game_date.strftime('%Y-%m-%d %H:%M:%S')


    query = """
    SELECT GAME_ID, GAME_DATE
    FROM GAMES 
    WHERE H_TEAM_ID = ? 
    AND A_TEAM_ID = ? 
    AND GAME_DATE BETWEEN ? AND ?
    """
    
    result = pd.read_sql_query(query, conexion, params=[home_team_id, away_team_id, start_date, game_date])
    
    if not result.empty:
        if len(result) > 1:
            game_ids.append(result.iloc[-1]['GAME_ID'])
            #if result['GAME_DATE'].min == game_date:
            #    game_ids.append(result.loc[result['GAME_DATE'] == game_date]['GAME_ID'])
            #else:
            #    game_ids.append(result.iloc[result['GAME_DATE'] == start_date]['GAME_ID'])
        else:
            game_ids.append(result.iloc[0]['GAME_ID'])
    else:
        game_ids.append(None)

# Añadir los GAME_IDs al DataFrame dataset_ods
dataset_ods['GAME_ID'] = game_ids

# Cerrar la conexión
conexion.close()

In [41]:
conexion = sqlite3.connect("NBA_DATA.db") 
cursor = conexion.cursor()
cursor.execute("""
create table If Not Exists GAME_ODS (
    GAME_ID text,
    Average_1 double precision,
    Average_X double precision, 
    Average_2 double precision, 
    Highest_1 double precision, 
    Highest_X double precision, 
    Highest_2 double precision,
    Average_H double precision, 
    Average_A double precision, 
    Highest_H double precision, 
    Highest_A double precision, 
    Average_1X double precision,
    Average_12 double precision, 
    Average_X2 double precision, 
    Highest_1X double precision, 
    Highest_12 double precision, 
    Highest_X2 double precision,
    primary key (GAME_ID)
    foreign key (GAME_ID) REFERENCES GAMES(GAME_ID)
)
""")

dataset_guardar = dataset_ods[['GAME_ID', 'Average_1', 'Average_X', 'Average_2', 'Highest_1', 'Highest_X', 'Highest_2', 'Average_H', 'Average_A', 'Highest_H', 'Highest_A', 'Average_1X', 'Average_12', 'Average_X2', 'Highest_1X', 'Highest_12', 'Highest_X2']]
dataset_guardar.to_sql('GAME_ODS', conexion, if_exists='append', index=False)

1377

In [47]:
from generate_database import database_inicialization
database_inicialization()

Generation season schedule and game logs for season 2017
scheduleFrame takes: 5:9
0
GameLogs takes: 0:0
Generation season schedule and game logs for season 2018
scheduleFrame takes: 5:10
0
GameLogs takes: 0:0
Generation season schedule and game logs for season 2019
scheduleFrame takes: 5:10
0
GameLogs takes: 0:0
Generation season schedule and game logs for season 2020
scheduleFrame takes: 5:10
0
GameLogs takes: 0:0
Generation season schedule and game logs for season 2021
scheduleFrame takes: 5:8
0
GameLogs takes: 0:0
Generation season schedule and game logs for season 2022
scheduleFrame takes: 6:15
0
GameLogs takes: 0:0
Generation season schedule and game logs for season 2023
scheduleFrame takes: 5:11
0
GameLogs takes: 0:0
Generando tabla de odds
Generando tabla de odds para la temporada results
Generando tabla de urls
https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/
https://www.oddsportal.com/basketball/usa/nba-2023-2024/results//#/page/1/
https://www.oddsportal.com/ba

KeyboardInterrupt: 

In [9]:
x = "https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/2/"
x = x.split('/')[5].split('-')[1:]
x = x[0] + '-'+x[1][2:]
x


'2023-24'

In [3]:
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from datetime import timedelta, datetime
from maper import EQUIPOS_ODDS

In [213]:

def scroll_to_bottom(driver, pause_time=2):
    """Desplazarse poco a poco hasta el final de la página."""
    last_height = driver.execute_script("return document.body.scrollHeight*0.9")
    
    while True:
        # Desplazarse hacia abajo
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*0.9);")
        
        # Esperar a que se cargue el contenido
        time.sleep(pause_time)
        
        # Calcular la nueva altura de la página y comparar con la última altura
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def get_urls(url, pages):
    partidos = []
    home_team = []
    away_team = []
    fechas = []
    i = 0

    
    print(url)
    temp_urls = [url+f'/#/page/{j}/' for j in range(1,pages+1)]

    options = Options()
    #options.add_experimental_option("detach", True)
    options.add_argument("--disable-search-engine-choice-screen")

    j = 0
    while j <= pages-1:
        print(temp_urls[j])
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
        driver.get(temp_urls[j])

        driver.maximize_window()
        time.sleep(5)
        scroll_to_bottom(driver, pause_time=3)
        time.sleep(5)
        html = driver.page_source
        
        soup = BeautifulSoup(html, 'html.parser')
        
        divs = soup.find_all('div', class_='eventRow flex w-full flex-col text-xs')

        for div in divs:
            
            fecha_div = div.find('div', class_='text-black-main font-main w-full truncate text-xs font-normal leading-5')
            
            if fecha_div is not None:
                fecha_div = fecha_div.text
                fecha = fecha_div.split('-')[0].rstrip()
                if 'Today' in fecha:
                    fecha = datetime.now().strftime("%d %b %Y")
                elif 'Yesterday' in fecha:
                    fecha = (datetime.now() - timedelta(days=1)).strftime("%d %b %Y")
            divs2 = div.find_all('a', class_='next-m:flex next-m:!mt-0 ml-2 mt-2 min-h-[32px] w-full hover:cursor-pointer')

            for div2 in divs2:
                
                partidos.append('https://www.oddsportal.com' + div2['href'])
                equipos = div2.find_all('p', class_='participant-name truncate')
                home_team.append(equipos[0].text)
                away_team.append(equipos[1].text)
                fechas.append(fecha)
        j += 1        
        driver.quit()    

    temp_data_df = pd.DataFrame()       
    temp_data_df['URL'] = partidos
    temp_data_df['H_TEAM_NICKNAME'] = home_team
    temp_data_df['A_TEAM_NICKNAME'] = away_team
    temp_data_df['GAME_DATE'] = fechas

    return temp_data_df

def get_game_id(dataset_urls: pd.DataFrame):
    dataset_urls['H_TEAM_ID'] = dataset_urls['H_TEAM_NICKNAME'].map(EQUIPOS_ODDS).replace({float('nan'): None})
    dataset_urls['A_TEAM_ID'] = dataset_urls['A_TEAM_NICKNAME'].map(EQUIPOS_ODDS).replace({float('nan'): None})
    dataset_urls['GAME_DATE'] = pd.to_datetime(dataset_urls['GAME_DATE'])

    conexion = sqlite3.connect("NBA_DATA.db")

    # Crear una lista para almacenar los GAME_IDs
    game_ids = []
    # Realizar una consulta SQL para cada fila en dataset_urls
    for index, row in dataset_urls.iterrows():
        if row['H_TEAM_ID'] is None or row['A_TEAM_ID'] is None:
            game_ids.append(None)
            continue
        home_team_id = row['H_TEAM_ID']
        away_team_id = row['A_TEAM_ID']
        game_date = row['GAME_DATE']
        start_date = (game_date - pd.Timedelta(days=1)).strftime('%Y-%m-%d %H:%M:%S')
        game_date = game_date.strftime('%Y-%m-%d %H:%M:%S')


        query = """
        SELECT GAME_ID, GAME_DATE
        FROM GAMES 
        WHERE H_TEAM_ID = ? 
        AND A_TEAM_ID = ? 
        AND GAME_DATE BETWEEN ? AND ?
        """
        
        result = pd.read_sql_query(query, conexion, params=[home_team_id, away_team_id, start_date, game_date])
        
        if not result.empty:
            result.sort_values(by='GAME_DATE', ascending=False, inplace=True)
            if len(result) > 1:
                game_ids.append(result.loc[result["GAME_DATE"] == game_date]['GAME_ID'])
            else:
                game_ids.append(result.iloc[0]['GAME_ID'])
        else:
            game_ids.append(None)
    dataset_urls['GAME_ID'] = game_ids

    return dataset_urls[['URL', 'H_TEAM_NICKNAME', 'A_TEAM_NICKNAME', 'GAME_DATE', 'GAME_ID']]

def get_season(url: str):
    conexion = sqlite3.connect("NBA_DATA.db")
    season = url.split('/')[5].split('-')[1:]
    season = season[0] + '-'+season[1][2:]

    query = """
        SELECT GAME_ID
        FROM GAMES
        WHERE SEASON = ?
        """
    
    return pd.read_sql_query(query, conexion, params=[season])

def get_odds(dataset_urls: pd.DataFrame):

    keys = ['URL', 'GAME_ID', 'H_TEAM_NICKNAME', 'A_TEAM_NICKNAME', 'GAME_DATE', 'Average_1','Average_X','Average_2','Highest_1','Highest_X','Highest_2', 'Average_H', 'Average_A', 'Highest_H', 'Highest_A', 'Average_1X','Average_12','Average_X2','Highest_1X','Highest_12','Highest_X2']
    extensiones = ['#1X2;3', '#home-away;1', '#double;3']
    columnas = [['Average_1','Average_X','Average_2','Highest_1','Highest_X','Highest_2'],['Average_H', 'Average_A', 'Highest_H', 'Highest_A'],
            ['Average_1X','Average_12','Average_X2','Highest_1X','Highest_12','Highest_X2']]
    conexion = sqlite3.connect("NBA_DATA.db") 

    dataset_urls = get_game_id(dataset_urls)
    ids = list(get_season(dataset_urls.iloc[-1]['URL']))
    filtered_dataset_urls = dataset_urls[~dataset_urls['GAME_ID'].isin(ids)]
    filtered_dataset_urls = dataset_urls[(~dataset_urls['GAME_ID'].isin(ids)) & (dataset_urls['GAME_ID'].notna())]
    
    options = Options()
    #options.add_experimental_option("detach", True)
    options.add_argument("--disable-search-engine-choice-screen")
    options.add_argument("--headless")
    
    dataset_ods = pd.DataFrame(columns=keys)

    for index, row in filtered_dataset_urls.iterrows():

        print(index)
        diccionario_estadisticas = {key: None for key in keys}
        diccionario_estadisticas['URL'] = row['URL']
        diccionario_estadisticas['H_TEAM_NICKNAME'] = row['H_TEAM_NICKNAME']
        diccionario_estadisticas['A_TEAM_NICKNAME'] = row['A_TEAM_NICKNAME']
        diccionario_estadisticas['GAME_DATE'] = row['GAME_DATE']
        diccionario_estadisticas['GAME_ID'] = row['GAME_ID']
        i = -1
        for extension in extensiones:
            
            try:
                i += 1
                url = row['URL'] + extension

                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)

                driver.get(url)
                driver.maximize_window()
                time.sleep(3)
                last_height = driver.execute_script("return document.body.scrollHeight")
                while True:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(3)  # Espera un poco para que se cargue el contenido dinámico
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height
                    
                html = driver.page_source

                soup = BeautifulSoup(html, 'html.parser')

                divs = soup.find_all('div', class_='border-black-borders flex h-9 border-b border-l border-r text-xs')

                for div in divs:
                    ps = div.find_all('p', class_='height-content')
                    calculator_list = [[] for _ in range(len(ps)-1)]
                    for i in range(len(ps)-1):
                        calculator_list[i].append(float(ps[i+1].text))
                from statistics import mean
                medias = [mean(lista) for lista in calculator_list]

                # Aplicar la transformación según la condición
                for j in range(len(medias)):
                    diccionario_estadisticas[columnas[i][j]] = medias[j]
            except Exception as e:
                # Si ocurre un error, se captura la excepción y se imprime la variable
                print(f"Ocurrió un error: {e} en el partido {index}")

        data_df = pd.DataFrame([diccionario_estadisticas])
        #data_df[['GAME_ID', 'Average_1', 'Average_X', 'Average_2', 'Highest_1', 'Highest_X', 'Highest_2', 'Average_H', 'Average_A', 'Highest_H', 'Highest_A', 'Average_1X', 'Average_12', 'Average_X2', 'Highest_1X', 'Highest_12', 'Highest_X2']].to_sql('GAME_ODS', conexion, if_exists='append', index=False)
        # Aqui debo incluir el formateo para que todo funcione correctamente
        dataset_ods = pd.concat([dataset_ods, data_df], ignore_index=True)
    return dataset_ods



def process_odds(dataset_ods: pd.DataFrame):
    conexion = sqlite3.connect("NBA_DATA.db")

    query = """
    SELECT GAME_ID
    FROM GAME_ODS
    """
    result = pd.read_sql_query(query, conexion)
    dataset_ods = dataset_ods[~dataset_ods['GAME_ID'].isin(list(result['GAME_ID']))]

    # Cerrar la conexión
    conexion.close()
    return dataset_ods[['GAME_ID', 'Average_1', 'Average_X', 'Average_2', 'Highest_1', 'Highest_X', 'Highest_2', 'Average_H', 'Average_A', 'Highest_H', 'Highest_A', 'Average_1X', 'Average_12', 'Average_X2', 'Highest_1X', 'Highest_12', 'Highest_X2']]

In [214]:
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from datetime import timedelta, datetime
from maper import EQUIPOS_ODDS
import sqlite3

urls = [#'https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/',
            #'https://www.oddsportal.com/basketball/usa/nba-2022-2023/results/',
            #'https://www.oddsportal.com/basketball/usa/nba-2021-2022/results/',
            #'https://www.oddsportal.com/basketball/usa/nba-2020-2021/results/',
            'https://www.oddsportal.com/basketball/usa/nba-2019-2020/results/',
            'https://www.oddsportal.com/basketball/usa/nba-2018-2019/results/',
            'https://www.oddsportal.com/basketball/usa/nba-2017-2018/results/',]
pages = [28,
            28,
            28,
            28,
            28,
            28,
            28,]
page = 28

for url in urls:
        print(f"Generando tabla de odds para la temporada {url.split('/')[-2]}")
        conexion = sqlite3.connect("NBA_DATA.db") 

        print("Generando tabla de urls")
        dataset_urls = get_urls(url, page)
        print("Generando tabla de odds")
        dataset_ods = get_odds(dataset_urls)
        dataset_guardar = process_odds(dataset_ods)

        dataset_guardar.to_sql('GAME_ODS', conexion, if_exists='append', index=False)
        print("Tabla de odds guardada")
        

Generando tabla de odds para la temporada results
Generando tabla de urls
https://www.oddsportal.com/basketball/usa/nba-2019-2020/results/
https://www.oddsportal.com/basketball/usa/nba-2019-2020/results//#/page/1/
https://www.oddsportal.com/basketball/usa/nba-2019-2020/results//#/page/2/
https://www.oddsportal.com/basketball/usa/nba-2019-2020/results//#/page/3/
https://www.oddsportal.com/basketball/usa/nba-2019-2020/results//#/page/4/
https://www.oddsportal.com/basketball/usa/nba-2019-2020/results//#/page/5/
https://www.oddsportal.com/basketball/usa/nba-2019-2020/results//#/page/6/
https://www.oddsportal.com/basketball/usa/nba-2019-2020/results//#/page/7/
https://www.oddsportal.com/basketball/usa/nba-2019-2020/results//#/page/8/
https://www.oddsportal.com/basketball/usa/nba-2019-2020/results//#/page/9/
https://www.oddsportal.com/basketball/usa/nba-2019-2020/results//#/page/10/
https://www.oddsportal.com/basketball/usa/nba-2019-2020/results//#/page/11/
https://www.oddsportal.com/basketb

C:\Users\andre\AppData\Local\Temp\ipykernel_5920\1314240248.py:210: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset_ods = pd.concat([dataset_ods, data_df], ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
40
41
42
44
45
46
47
48
49
51
52
53
54
56
57
58
59
60
61
62
63
64
66
67
68
69
70
71
72
74
75
76
77
78
79
80
81
84
85
86
87
88
91
92
93
94
95
96
97
98
99
101
102
103
104
105
106
107
108
110
111
112
113
114
116
117
118
119
120
122
123
125
126
127
129
130
131
132
133
135
136
137
138
139
141
142
143
144
145
146
148
150
151
152
153
154
155
156
157
158
159
160
163
164
165
166
167
168
169
170
207
208
209
210
211
212
213
214
215
216
217
218
219
220
222
223
224
225
226
227
228
229
230
231
233
234
235
236
237
239
240
241
242
244
245
247
248
249
250
251
252
253
254
256
257
258
260
261
262
263
264
265
266
268
269
270
271
272
273
274
276
277
278
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
296
297
298
300
301
302
303
304
305
306
308
309
310
311
312
313
314
315
316
318
319
320
321
322
323
325
326
327
328
329
330
331
333
334
335
336
337
339
340
341
342
343
344
345
348
349
350
351
35

C:\Users\andre\AppData\Local\Temp\ipykernel_5920\1314240248.py:210: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset_ods = pd.concat([dataset_ods, data_df], ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
46
47
48
49
51
52
53
54
55
57
58
59
60
61
62
63
64
65
67
68
69
70
71
72
73
75
76
77
78
79
80
81
82
84
85
86
87
88
89
90
91
92
93
94
96
97
98
100
101
102
103
105
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
125
126
127
128
129
130
131
132
133
134
135
136
138
139
140
142
143
144
145
146
147
148
149
150
151
152
154
155
156
157
158
159
160
161
162
163
164
166
167
168
169
170
171
172
173
174
175
176
177
179
180
181
182
183
184
185
187
188
189
190
192
193
194
195
196
197
198
200
201
202
203
204
205
206
208
209
210
211
212
214
215
216
217
218
219
220
222
223
224
225
226
227
228
229
230
231
232
233
234
235
237
238
239
240
241
242
243
245
247
248
249
250
251
252
253
254
255
257
258
259
260
261
263
264
265
266
267
268
269
270
271
273
274
275
277
278
279
280
281
282
284
285
286
287
289
290
291
292
294
295
296
297
298
299
300
301
302
304
305
306
308
309
310


C:\Users\andre\AppData\Local\Temp\ipykernel_5920\1314240248.py:210: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset_ods = pd.concat([dataset_ods, data_df], ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
21
22
25
26
29
30
33
34
36
38
40
41
42
43
45
46
47
48
49
51
52
53
54
55
57
59
60
61
62
63
65
66
68
70
71
72
73
74
76
77
79
80
81
82
84
85
86
87
88
89
91
92
93
94
96
97
98
99
101
102
103
104
105
106
107
108
109
111
112
113
114
115
116
118
119
120
121
122
124
125
126
127
128
129
130
131
132
134
135
136
137
138
140
141
142
143
144
145
146
148
149
150
151
152
153
154
155
156
157
158
159
160
162
163
164
166
167
168
169
170
171
172
173
174
175
176
178
180
181
182
183
184
185
186
187
188
189
190
192
193
194
195
196
197
198
199
200
201
203
204
205
206
207
208
209
211
212
213
214
215
216
217
218
219
221
222
223
224
225
226
228
229
230
231
232
233
234
235
237
239
240
241
242
243
245
246
247
248
249
250
252
253
255
256
257
258
259
260
261
262
263
264
265
267
268
269
271
272
273
274
275
277
278
279
280
281
282
283
284
285
286
289
290
291
292
293
294
295
296
297
298
299
301
302
304
305
306
307
308
309
310
311
312
313
314
315
316
317
319
320
321
322
323
3

In [195]:
dataset_guardar['GAME_ID'].duplicated().sum()

0

In [196]:
subset = dataset_guardar2[dataset_guardar2.duplicated(subset=['GAME_ID'], keep=False)].sort_values(by='GAME_ID', ascending=True)
subset

,URL,GAME_ID,H_TEAM_NICKNAME,A_TEAM_NICKNAME,GAME_DATE,Average_1,Average_X,Average_2,Highest_1,Highest_X,...,Highest_H,Highest_A,Average_1X,Average_12,Average_X2,Highest_1X,Highest_12,Highest_X2,H_TEAM_ID,A_TEAM_ID


In [193]:
def get_game_id(dataset_urls: pd.DataFrame):
    dataset_urls['H_TEAM_ID'] = dataset_urls['H_TEAM_NICKNAME'].map(EQUIPOS_ODDS).replace({float('nan'): None})
    dataset_urls['A_TEAM_ID'] = dataset_urls['A_TEAM_NICKNAME'].map(EQUIPOS_ODDS).replace({float('nan'): None})
    dataset_urls['GAME_DATE'] = pd.to_datetime(dataset_urls['GAME_DATE'])

    conexion = sqlite3.connect("NBA_DATA.db")

    # Crear una lista para almacenar los GAME_IDs
    game_ids = []
    # Realizar una consulta SQL para cada fila en dataset_urls
    for index, row in dataset_urls.iterrows():
        if row['H_TEAM_ID'] is None or row['A_TEAM_ID'] is None:
            game_ids.append(None)
            continue
        home_team_id = row['H_TEAM_ID']
        away_team_id = row['A_TEAM_ID']
        game_date = row['GAME_DATE']
        start_date = (game_date - pd.Timedelta(days=1)).strftime('%Y-%m-%d %H:%M:%S')
        game_date = game_date.strftime('%Y-%m-%d %H:%M:%S')


        query = """
        SELECT GAME_ID, GAME_DATE
        FROM GAMES 
        WHERE H_TEAM_ID = ? 
        AND A_TEAM_ID = ? 
        AND GAME_DATE BETWEEN ? AND ?
        """
        
        result = pd.read_sql_query(query, conexion, params=[home_team_id, away_team_id, start_date, game_date])
        
        if not result.empty:
            result.sort_values(by='GAME_DATE', ascending=False, inplace=True)
            if len(result) > 1:
                game_ids.append(result.loc[result["GAME_DATE"] == game_date]['GAME_ID'])
            else:
                game_ids.append(result.iloc[0]['GAME_ID'])
        else:
            game_ids.append(None)
    dataset_urls['GAME_ID'] = game_ids

    return dataset_urls

In [192]:
conexion = sqlite3.connect("NBA_DATA.db")
query = """
        SELECT GAME_ID, GAME_DATE
        FROM GAMES 
        WHERE H_TEAM_ID = ? 
        AND A_TEAM_ID = ? 
        AND GAME_DATE BETWEEN ? AND ?
        """
result = pd.read_sql_query(query, conexion, params=[1610612764, 1610612753, '2020-12-26 00:00:00', '2020-12-27 00:00:00'])
result = pd.read_sql_query(query, conexion, params=[1610612759, 1610612756, '2021-05-15 00:00:00', '2021-05-16 00:00:00'])
result.sort_values(by='GAME_DATE', ascending=False, inplace=True)
result.loc[result["GAME_DATE"] == '2021-05-16 00:00:00']

,GAME_ID,GAME_DATE
0,0022001078,2021-05-16 00:00:00


In [208]:
dataset_guardar = get_game_id(dataset_ods)
dataset_guardar

,URL,GAME_ID,H_TEAM_NICKNAME,A_TEAM_NICKNAME,GAME_DATE,Average_1,Average_X,Average_2,Highest_1,Highest_X,...,Highest_H,Highest_A,Average_1X,Average_12,Average_X2,Highest_1X,Highest_12,Highest_X2,H_TEAM_ID,A_TEAM_ID
0,https://www.oddsportal.com/basketball/usa/nba-...,0042000406,Milwaukee Bucks,Phoenix Suns,2021-07-21,NaN,NaN,NaN,None,None,...,None,None,1.50,1.03,2.20,None,None,None,1610612749,1610612756
1,https://www.oddsportal.com/basketball/usa/nba-...,0042000405,Phoenix Suns,Milwaukee Bucks,2021-07-18,NaN,NaN,NaN,None,None,...,None,None,1.50,1.04,2.20,None,None,None,1610612756,1610612749
2,https://www.oddsportal.com/basketball/usa/nba-...,0042000404,Milwaukee Bucks,Phoenix Suns,2021-07-15,NaN,NaN,NaN,None,None,...,None,None,1.48,1.03,2.20,None,None,None,1610612749,1610612756
3,https://www.oddsportal.com/basketball/usa/nba-...,0042000403,Milwaukee Bucks,Phoenix Suns,2021-07-12,NaN,NaN,NaN,None,None,...,None,None,1.40,1.03,2.50,None,None,None,1610612749,1610612756
4,https://www.oddsportal.com/basketball/usa/nba-...,0042000402,Phoenix Suns,Milwaukee Bucks,2021-07-09,NaN,NaN,NaN,None,None,...,None,None,1.50,1.03,2.20,None,None,None,1610612756,1610612749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,https://www.oddsportal.com/basketball/usa/nba-...,0022000011,Indiana Pacers,New York Knicks,2020-12-24,NaN,NaN,NaN,None,None,...,None,None,1.38,13.50,3.55,None,None,None,1610612754,1610612752
1009,https://www.oddsportal.com/basketball/usa/nba-...,0022000012,Orlando Magic,Miami Heat,2020-12-24,NaN,NaN,NaN,None,None,...,None,None,2.75,13.25,1.56,None,None,None,1610612753,1610612748
1010,https://www.oddsportal.com/basketball/usa/nba-...,0022000013,Philadelphia 76ers,Washington Wizards,2020-12-24,NaN,NaN,NaN,None,None,...,None,None,1.33,3.50,2.90,None,None,None,1610612755,1610612764
1011,https://www.oddsportal.com/basketball/usa/nba-...,0022000002,Los Angeles Lakers,Los Angeles Clippers,2020-12-23,NaN,NaN,NaN,None,None,...,None,None,1.80,14.00,2.15,None,None,None,1610612747,1610612746


In [209]:

dataset_guardar['GAME_ID'] = dataset_guardar['GAME_ID'].astype(str)

object_cols_to_float = [col for col in dataset_guardar.columns if dataset_guardar[col].dtype == 'object' and col != 'GAME_ID']

# Convertir las columnas object restantes a float, manejando posibles errores
for col in object_cols_to_float:
    # Usar pd.to_numeric con errors='coerce' para convertir a float
    # y reemplazar los valores no convertibles con NaN
    dataset_guardar[col] = pd.to_numeric(dataset_guardar[col], errors='coerce')
print(dataset_guardar.dtypes)

URL                       float64
GAME_ID                    object
H_TEAM_NICKNAME           float64
A_TEAM_NICKNAME           float64
GAME_DATE          datetime64[ns]
Average_1                 float64
Average_X                 float64
Average_2                 float64
Highest_1                 float64
Highest_X                 float64
Highest_2                 float64
Average_H                 float64
Average_A                 float64
Highest_H                 float64
Highest_A                 float64
Average_1X                float64
Average_12                float64
Average_X2                float64
Highest_1X                float64
Highest_12                float64
Highest_X2                float64
H_TEAM_ID                   int64
A_TEAM_ID                   int64
dtype: object


In [210]:
conexion = sqlite3.connect("NBA_DATA.db")
dataset_guardar = dataset_guardar[['GAME_ID', 'Average_1', 'Average_X', 'Average_2', 'Highest_1', 'Highest_X', 'Highest_2', 'Average_H', 'Average_A', 'Highest_H', 'Highest_A', 'Average_1X', 'Average_12', 'Average_X2', 'Highest_1X', 'Highest_12', 'Highest_X2']]

dataset_guardar.to_sql('GAME_ODS', conexion, if_exists='append', index=False)

1013

In [53]:
conexion = sqlite3.connect("NBA_DATA.db")

query = """
        SELECT GAME_ID, GAME_DATE
        FROM GAMES 
        WHERE H_TEAM_ID = ? 
        AND A_TEAM_ID = ? 
        AND GAME_DATE BETWEEN ? AND ?
        """
        
result = pd.read_sql_query(query, conexion, params=[1610612759, 1610612756, '2021-05-15 00:00:00', '2021-05-16 00:00:00'])
result.sort_values(by='GAME_DATE', ascending=False, inplace=True)
result


,GAME_ID,GAME_DATE
0,0022001078,2021-05-16 00:00:00
1,0022001065,2021-05-15 00:00:00


In [58]:
duplicados = dataset_ods2[dataset_ods2['GAME_ID'].duplicated(keep=False)]
duplicados

,URL,GAME_ID,H_TEAM_NICKNAME,A_TEAM_NICKNAME,GAME_DATE,Average_1,Average_X,Average_2,Highest_1,Highest_X,...,Highest_H,Highest_A,Average_1X,Average_12,Average_X2,Highest_1X,Highest_12,Highest_X2,H_TEAM_ID,A_TEAM_ID


In [57]:
dataset_ods2.at[83, 'GAME_ID'] = '0022001078'

In [22]:
dataset_ods_copy = dataset_ods.copy()

In [23]:
dataset_ods_copy['H_TEAM_ID'] = dataset_ods_copy['H_TEAM_NICKNAME'].map(EQUIPOS_ODDS).replace({float('nan'): None})
dataset_ods_copy['A_TEAM_ID'] = dataset_ods_copy['A_TEAM_NICKNAME'].map(EQUIPOS_ODDS).replace({float('nan'): None})
dataset_ods_copy['GAME_DATE'] = pd.to_datetime(dataset_ods_copy['GAME_DATE'])

conexion = sqlite3.connect("NBA_DATA.db")

# Mos quedamos solo con los datos nuevos
min_date = dataset_ods_copy['GAME_DATE'].min().strftime('%Y-%m-%d %H:%M:%S')
max_date = dataset_ods_copy['GAME_DATE'].max().strftime('%Y-%m-%d %H:%M:%S')
query = """
SELECT GAME_ID
FROM GAME_ODS
"""
result = pd.read_sql_query(query, conexion)
dataset_ods_copy = dataset_ods_copy[~dataset_ods_copy['GAME_ID'].isin(result['GAME_ID'])]

# Cerrar la conexión
conexion.close()
dataset_ods_copy[['GAME_ID', 'Average_1', 'Average_X', 'Average_2', 'Highest_1', 'Highest_X', 'Highest_2', 'Average_H', 'Average_A', 'Highest_H', 'Highest_A', 'Average_1X', 'Average_12', 'Average_X2', 'Highest_1X', 'Highest_12', 'Highest_X2']]


,GAME_ID,Average_1,Average_X,Average_2,Highest_1,Highest_X,Highest_2,Average_H,Average_A,Highest_H,Highest_A,Average_1X,Average_12,Average_X2,Highest_1X,Highest_12,Highest_X2
0,0022300887,1.89,15.00,1.95,1.95,15.00,1.95,1.90,1.92,1.96,1.95,1.83,1.04,1.77,1.83,1.04,1.77
1,0022300888,1.28,17.00,3.78,1.29,17.00,4.00,1.15,5.63,1.16,5.77,1.24,1.03,3.50,1.24,1.03,3.50
2,0022300889,1.83,15.00,2.03,1.83,15.00,2.10,1.80,2.05,1.81,2.06,1.69,1.04,1.87,1.69,1.04,1.87
4,0022300885,2.90,15.50,1.43,3.00,16.00,1.44,3.85,1.29,4.00,1.29,2.75,1.03,1.35,2.75,1.03,1.35
5,0022300884,1.94,15.00,1.90,1.95,15.00,1.95,1.95,1.87,1.97,1.89,1.77,1.04,1.83,1.77,1.04,1.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,0022300064,1.61,15.50,2.40,1.63,16.00,2.50,1.52,2.64,1.53,2.76,1.50,1.04,2.25,1.50,1.04,2.25
909,0022300065,2.34,15.00,1.64,2.40,15.00,1.67,2.46,1.58,2.54,1.61,2.10,1.04,1.56,2.10,1.04,1.56
910,0022300066,1.65,15.50,2.33,1.67,16.00,2.40,1.55,2.55,1.57,2.68,1.50,1.04,2.25,1.50,1.04,2.25
911,0022300062,1.71,15.00,2.21,1.74,15.00,2.25,1.69,2.23,1.73,2.30,1.63,1.04,2.00,1.63,1.04,2.00


In [26]:
import sqlite3

# Conectar a la base de datos SQLite
conexion = sqlite3.connect("NBA_DATA.db")

# Crear un cursor
cursor = conexion.cursor()

# Eliminar todo el contenido de la tabla GAME_ODS
cursor.execute("DELETE FROM GAME_ODS")

# Confirmar los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()

print("El contenido de la tabla GAME_ODS ha sido eliminado.")

El contenido de la tabla GAME_ODS ha sido eliminado.


In [75]:
import requests

url = "https://jsonjames-jsonodds-v1.p.rapidapi.com/api/sports"

headers = {"x-rapidapi-host": "jsonjames-jsonodds-v1.p.rapidapi.com"}

response = requests.get(url, headers=headers)

print(response.json())

{'message': 'Invalid API key. Go to https://docs.rapidapi.com/docs/keys for more info.'}
